In [1]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-83jvh6vp
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-83jvh6vp
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11756 sha256=f60f64293d9685c5f3584540fd7768633ad793555689378975586eb1f72fe3ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-buvnfekp/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [2]:
!pip install textcleaner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for textcleaner: filename=textcleaner-0.4.26-py3-none-any.whl size=4737 sha256=2f52e09a6afadbe8f757d1c8659f6e73532b0793688ddfd0a2d380705697a8f4
  Stored in directory: /root/.cache/pip/wheels/67/e1/1a/8a3dc9c64347e08039c1e69924c9af6ba0a395a0ed92cfb493
Successfully built textcleaner


In [3]:
import pandas as pd
import requests
import numpy as np
import preprocess_kgptalkie as kgp
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

In [8]:
data=pd.read_csv("/content/drive/MyDrive/libros/libros_v2.csv")

In [9]:
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,url,target,Unnamed: 6,Unnamed: 7,posibles targets,target.1
0,NaN,NaN,NaN,NaN,https://www.gutenberg.org/cache/epub/68468/pg6...,[0],NaN,NaN,0.0,humor
1,NaN,NaN,NaN,NaN,https://www.gutenberg.org/cache/epub/63429/pg6...,"[0,3]",NaN,NaN,1.0,amor
2,NaN,NaN,NaN,NaN,https://www.gutenberg.org/cache/epub/15247/pg1...,[0],NaN,NaN,3.0,science fiction
3,NaN,NaN,NaN,NaN,https://www.gutenberg.org/cache/epub/47087/pg4...,[4],NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,https://www.gutenberg.org/cache/epub/18857/pg1...,"[3,4]",NaN,NaN,4.0,aventuras


In [10]:
data.shape

(370, 10)

In [11]:
df=pd.DataFrame()

In [12]:
df["url"]=data["url"]
df["target"]=data["target"]

In [13]:
df.head()

,url,target
0,https://www.gutenberg.org/cache/epub/68468/pg6...,[0]
1,https://www.gutenberg.org/cache/epub/63429/pg6...,"[0,3]"
2,https://www.gutenberg.org/cache/epub/15247/pg1...,[0]
3,https://www.gutenberg.org/cache/epub/47087/pg4...,[4]
4,https://www.gutenberg.org/cache/epub/18857/pg1...,"[3,4]"


In [14]:
def get_text(df,columna,nueva_columna):
  df[nueva_columna]=np.nan
  n=df.shape[0]
  for i in range(n):
    response = requests.get(df[columna][i])
    df[nueva_columna][i]=response.text

def get_clean(x):
    x = str(x).lower().replace('\\', ' ').replace('\r\n', ' '  )
    x = kgp.cont_exp(x)
    x = kgp.remove_emails(x)
    x = kgp.remove_urls(x)
    x = kgp.remove_html_tags(x)
    x = kgp.remove_accented_chars(x)
    x = kgp.remove_special_chars(x)
    x = kgp.remove_dups_char(x)


    return x

In [ ]:
get_text(df,"url","text")
df["text"]= df["text"].apply(lambda x: get_clean(x))

df['clean_text'] = df['text'].apply(lambda x: ' '.join([t for t in x.split() if t not in stopwords]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df.head()

,url,target,text,clean_text
0,https://www.gutenberg.org/cache/epub/68468/pg6...,[0],the project gutenberg ebook of told by the col...,project gutenberg ebook told colonel william l...
1,https://www.gutenberg.org/cache/epub/63429/pg6...,"[0,3]",the project gutenberg ebook of joe carsons wea...,project gutenberg ebook joe carsons weapon jam...
2,https://www.gutenberg.org/cache/epub/15247/pg1...,[0],the project gutenberg ebook of john henry smit...,project gutenberg ebook john henry smith frede...
3,https://www.gutenberg.org/cache/epub/6122/pg61...,[2],the project gutenberg ebook of tobogganing on ...,project gutenberg ebook tobogganing parnassus ...
4,https://www.gutenberg.org/cache/epub/26797/pg2...,[2],the project gutenberg ebook of something else ...,project gutenberg ebook franklin p adams ebook...


In [ ]:
df.to_csv("/content/drive/MyDrive/libros/prueba_final.csv")